In [15]:
import pandas as pd
import math
import requests
from bs4 import BeautifulSoup

In [1]:


#feature list
features = [
    "property name",
    "address",
    "price range",
    "room area",
    "bedroom",
    "bathroom",
    "description",
    "security level"
]

def main_crawler(starting_url):
    #the current main_crawler function now only returns the links of all properties
    """
    Running the mini crawler and returns the all property links starting from a url
    
    Input:
        starting_url: the url page where we start
    
    Output:
        all_retrieves: a list of all page link of properties
    """
    
    page_to_check = starting_url
    next_page_exists = True
    all_retrieves=[]
    
    while next_page_exists:
        page_to_check,apt_links = mini_crawler(page_to_check)
        all_retrieves += apt_links
        if page_to_check==None:
            next_page_exists=False
            
    return all_retrieves

def mini_crawler(url):
    """
    Extract all properties' link from current page
    
    Input：
        url: the url of the page we hope to check
        
    Output:
        link_next_page: the status of next page
        list_apt_link: a list of properties' link
    """
    
    #exstracting html information of current page
    headers ={"User-Agent" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"}
    response = requests.get(url, headers = headers, timeout = 300)
    soup = bs4.BeautifulSoup(response.text, "html5lib")
    
    #getting the link of next page
    next_page = soup.find("link",rel="next")
    if next_page == None:
        link_next_page=None
    else:
        link_next_page = soup.find("link",rel="next").get("href")
    
    #extracting all links of properties within current page
    list_apt_link = []
    houses_html = soup.find_all("li", class_="mortar-wrapper")
    for house in houses_html:
        apt_link = house.find("a").get('href')
        list_apt_link.append(apt_link)
        
    return link_next_page,list_apt_link

In [13]:
#load crime data
columns = ["CASE#", "LATITUDE", "LONGITUDE"]
crime = pd.read_csv("Crimes_Data.csv", usecols=columns)
crime = crime.dropna()
crime

CASE#           0
LATITUDE     2884
LONGITUDE    2884
dtype: int64


,CASE#,LATITUDE,LONGITUDE
2,JE266628,41.748486,-87.602675
3,JE266536,41.880661,-87.731186
5,JE267466,41.871540,-87.705839
6,JE266473,41.780851,-87.649674
7,JE267222,41.859989,-87.735995
...,...,...,...
205762,JE492545,41.902703,-87.629950
205763,JF103181,41.731960,-87.636205
205764,JF102974,41.904582,-87.714655
205765,JF103076,41.987483,-87.712672


In [22]:

#calculate number of crime incidents in the neighborhood\
#from website: https://www.geeksforgeeks.org/program-distance-two-points-earth/#:~:text=For%20this%20divide%20the%20values,is%20the%20radius%20of%20Earth.
def if_crime_in_neighborhood(lo1, la1, lo2, la2, radius):
    lo1 = math.radians(lo1)
    la1 = math.radians(la1)
    lo2 = math.radians(lo2)
    la2 = math.radians(la2)

    lo_dist = lo1 - lo2
    la_dist = la1 - la2

    a = math.sin(la_dist/2)**2 + math.cos(la1) * math.cos(la2) * math.sin(lo_dist/2)**2

    c = 2 * math.asin(math.sqrt(a))

    earth_r_in_mile = 3956

    if c * earth_r_in_mile <= radius:
        return True
    else:
        return False


def find_lo_range(lo1, la1):
    



In [ ]:
#data visualization